<a href="https://colab.research.google.com/github/luciengaitskell/surfacegrad/blob/master/multivarible-final_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Land Gradients
## Multivariable Final Project
### By Lucien Gaitskell '21
---

[Process Document](https://docs.google.com/document/d/1SkBM31T1R7t_NSwYksWuJIlWApiOJ3zho64ikw_aIK4/edit?usp=sharing)

Highlight flat regions and gradients in similar directions

Could be used to find:
- areas viable for houses
- bodies of water, rivers
- channels in mountains
- large roads

**NOTE:** Please read information about plotting in subsequent sections

In [0]:
import numpy as np


# 1/3 arcsecond data from USGS
DATA_URLS = {
        'RI': "https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/13/TIFF/n42w072/USGS_13_n42w072.tif",
        'CA': "https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/13/TIFF/n38w122/USGS_13_n38w122.tif",
        'SOCAL': "https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/13/TIFF/n37w119/USGS_13_n37w119.tif",
        'MORESOCAL': "https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/13/TIFF/n36w119/USGS_13_n36w119.tif",
}

## Supporting Code

### Data Retrieval / Processing

In [0]:
import re
import urllib.request
%pip install tqdm
from tqdm.notebook import tqdm

Note: you may need to restart the kernel to use updated packages.


In [0]:
def detect_latlon(res_url: str) -> (int, int):
    """
    Detect the reference latitude and longitude from a USGS resource url.
    """
    l_raw = re.search('[ns]\d\d[we]\d\d\d', res_url).group(0)
    if l_raw[0] == 'n':
        lat = 1
    elif l_raw[0] == 's':
        lat = -1

    lat *= int(l_raw[1:3])

    if l_raw[3] == 'e':
        lon = 1
    elif l_raw[3] == 'w':
        lon = -1

    lon *= int(l_raw[4:])
    return lat, lon

In [0]:
def download_file(res_url) -> str:
    """ Download file to temp location and return file path. """
    with tqdm(unit_scale=True) as pbar:
        def _dl_report(b, bsize, tsize):
            if tsize is not None:
                pbar.total = tsize
            pbar.update(b * bsize - pbar.n)
        fname, headers = urllib.request.urlretrieve(res_url, reporthook=_dl_report)
        return fname

### Gradients

In [0]:
def calc_gradient(A, axis):
    """ Calculate gradients at all points in a matrix along specified axis. """
    
    # Get first and last row / col and expand dimensions to match original
    if axis == 0:  # x
        prep = np.expand_dims(A[0, :], axis=0)  # [a,b,c,...] -> [[a,b,c,...]]
        app = np.expand_dims(A[-1, :], axis=0)
    elif axis == 1:  # y
        prep = np.expand_dims(A[:, 0], axis=1)  # [a,b,c,...] -> [[a],[b],[c],...]
        app = np.expand_dims(A[:, -1], axis=1)
    else:
        raise ValueError

    # Take difference with extra first and last row / col
    D = np.diff(A, axis=axis, prepend=prep, append=app)  # Take difference between adjacent elements

    # Take sum of adjacent rows / col
    if axis == 0:  # x
        B = D[:-1] + D[1:]
    elif axis == 1:  # y
        B = D[:, :-1] + D[:, 1:]

    return B / 2  # Complete formula



---


## Example

### Loading and Processing

Use my helper functions to download the specificied data from the known URL. The lat & lon is saved, and the TIFF file is downloaded, then loaded by `tifffile`.

In [0]:
%pip install tifffile
from tifffile import imread  # Used to read `tif` files from USGS

Note: you may need to restart the kernel to use updated packages.


The data is usually 250-500 MB, so this will take some time.

In [0]:
class Location:
    """ Contain geographic data of a specific location, and associate calculations """
    
    def __load_tiff(self, loc_name: str):
        """ Load data from remote TIF file. """
        url = DATA_URLS[loc_name]  # Grab URL from known sources based on name
        self.lat, self.lon = detect_latlon(url)  # Detect latitude and longitude from URL content
        self.orig_tiff = imread(download_file(url))  # Download file and load TIF directly
        
        # Ensure data is in standard range.
        ## Set 'empty' pixels to sea level:
        self.orig_tiff[self.orig_tiff < -10e20] = 0
        self.tif = self.orig_tiff.copy()
    
    def __create_xy(self):
        """ Create matching x, y 1D vectors for 2D z-axis data.  """
        self.y, self.x = np.arange(self.tif.shape[0]), np.arange(self.tif.shape[1])  # data in lat, lon
        self.x = self.x.astype('float64')
        self.y = self.y.astype('float64')
        
        # --- Scale given x,y values into meters --- #
        assert self.orig_tiff.shape[0] == self.orig_tiff.shape[1]  # Currently only supports same dims
        # Create scaling factor for converting single latitude step to meters
        self.SCALE = 1852 * 60 / self.orig_tiff.shape[0]  # (m / nmi) * (nmi / deg) / (num_steps)
        # @ max steps, num_steps == 1 deg latitude

        # Scale y values appropriately:
        self.y_m_orig = self.y * self.SCALE  # y * (nmi / deg) * (m / nmi)

        # Scale x values appropriately, with standard scale and including distortion
        self.x_m_orig = (self.x * self.SCALE  # x * (nmi / deg) * (m / nmi)
                         * np.cos(  # Use cosine for distortion component
                              np.radians(self.lat)  # Determine rad lat at corner point (approx)
                           )
                         )
    def __interp_xy(self):
        """ Create new set of x values at same spacing as latitude. """
        num_x = np.floor(np.max(self.x_m_orig) / self.SCALE)  # Determine maximum x value
        self.out_x = np.arange(0, num_x)  # Create even spaced range up to maximum
        
        x_map, y_map = np.meshgrid(self.out_x, self.y)  # Create final meshgrid for mapped points
        
        # -- Interpolate z values at new specified x meter spacing from existing meter values --- #
        # Operate over each latitude level to create new interpolated z values
        self.out_z = np.array([np.interp(
                                  self.out_x*self.SCALE,  # Point to map to (index times scaling)
                                  self.x_m_orig,     # X meter positions of existing z data
                                  self.tif[i]        # Existing/original z data
                                ) for i in range(self.x_m_orig.shape[0])])
        
    def __get_gradient(self):
        """ Use my gradient calculation to determine magnitude of gradient. """
        gradient = (calc_gradient(self.out_z, 0), calc_gradient(self.out_z, 1))  # vector

        self.gradient_mag = np.sqrt(np.square(gradient[0]) * np.square(gradient[1]))  # scalar
        self.gradient_dir = np.arctan2(*gradient)

    def __init__(self, loc_name, info=False):
        """ Create new location.
        
        @param loc_name (str): name of location to load
        @param info (bool, default=False): print state information while operating
        """
        self.__load_tiff(loc_name)
        if info: print("Scaling...")
        self.__create_xy()
        if info: print("Interpolating...")
        self.__interp_xy()
        if info: print("Gradient...")
        self.__get_gradient()

### Graphing

In [0]:
%pip install vispy
import vispy
from vispy import app, scene, color

Note: you may need to restart the kernel to use updated packages.


#### Interactive 3D
**Does not appear** to operate in Colab, however will display in local Jupyter session.

Provides interactive experience to view colored gradients based on angle. Can view full map region.

- Gradients are colored based on x/y angle, using HSV colorwheel
- Gradients are transparent above a threshold
- Vispy view control:
    - LMB: orbits the view around its center point.
    - RMB or scroll: change scale_factor (i.e. zoom level)
    - SHIFT + LMB: translate the center point
    - SHIFT + RMB: change FOV

**NOTE:** `InteractivePlot.show()` will require some time (<1 min) to populate image. The black box is NORMAL

In [0]:
class InteractivePlot:
    """ A Vispy based plotter for `Location`s. """
    
    def __init__(self, loc: Location):
        """ Create plot object
        
        @param loc (Location): The location to plot
        """
        
        self.canvas = None
        
        # Downsample data in order to plot full region faster (~100x)
        downsample = 10

        # Create new, smaller arrays:
        self.z_v = loc.out_z[::downsample, ::downsample]
        self.y_v = loc.y[::downsample]
        self.x_v = loc.out_x[::downsample]
        self.gd_dir = loc.gradient_dir[::downsample, ::downsample]
        self.gd_mag = loc.gradient_mag[::downsample, ::downsample]
        
        self.color_source = self.gd_dir  # Use gradient direction for color source

        # Scale color to max
        self.color_source = self.color_source/np.abs(np.max(self.color_source))

    def plot(self):
        """ Set up plot (canvas, camera, colormap, etc). """
        
        # Setup canvas and camera:
        self.canvas = scene.SceneCanvas(bgcolor='w')
        view = self.canvas.central_widget.add_view()
        view.camera = scene.TurntableCamera(up='z', fov=60)
        
        c = color.get_colormap('hsv').map(self.color_source).reshape(self.color_source.shape + (-1,))
        c[self.gd_mag > 0.3, 3] = 0  # Remove color from points of high gradient magnitude
        c = c.flatten().tolist()
        c=list(map(lambda x,y,z,w:(x,y,z,w), c[0::4],c[1::4],c[2::4],c[3::4]))
        
        # Create surface:
        p1 = scene.visuals.SurfacePlot(x=self.y_v, y=self.x_v, z=self.z_v)
        p1.mesh_data.set_vertex_colors(c)  # add colors
        view.add(p1)

        axis = scene.visuals.XYZAxis(parent=view.scene)  # Add reference axes
        
    def show(self):
        """
        Display plot!
        
        This will take some time (<30 sec) to display properly.
        Please zoom out once loaded to find the region.
        """
        self.canvas.show()
    
    @classmethod
    def from_loc_name(cls, loc_name):
        """ Run all basic methods for a given location name (simplifying usage). """
        print("Downloading '{}'".format(loc_name))
        loc = Location(loc_name, info=True)
        print("Setup plot @ {}, {} (lat, lon).".format(loc.lat, loc.lon))
        p = cls(loc)
        p.plot()
        p.show()

##### Example plots:
Please read notes above about timing.
If executing multiple at once, `.show()` will operate asyncronously, but will not function properly until all plots have sucessfully been displayed.

After starting... go get yourself some more coffee

In [0]:
InteractivePlot.from_loc_name("CA")


Scaling...
Interpolating...
Gradient...
Setup plot @ 38, -122 (lat, lon).


VispyWidget(height=600, width=800)

In [0]:
InteractivePlot.from_loc_name("SOCAL")


Scaling...
Interpolating...
Gradient...
Setup plot @ 37, -119 (lat, lon).


VispyWidget(height=600, width=800)

In [0]:
InteractivePlot.from_loc_name("MORESOCAL")


Scaling...
Interpolating...
Gradient...
Setup plot @ 36, -119 (lat, lon).


VispyWidget(height=600, width=800)

In [0]:
InteractivePlot.from_loc_name("RI")


Scaling...
Interpolating...
Gradient...
Setup plot @ 42, -72 (lat, lon).


VispyWidget(height=600, width=800)